<a href="https://colab.research.google.com/github/goya5858/custum-functions-for-kaggle/blob/main/Github_Clone_Push_file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<p><font size="5">基本的に、 １セル目の</p>

<p><font size="7" color="orange"> >> REPO_NAME</p>
<p><font size="5">さえ書き換えれば後はセルの実行だけで上手くいきます</p>

<p><font size="3">Githubのusername, email, passはJSONファイルに書いて適切なフォルダに配置してください</p>
<p><font size="3">想定ディレクトリは</p>    
<p><font size="3">/content/drive/MyDrive/kaggle/works/  <br>
|-- .ssh<br>
|　　+-- pemファイル類　このファイルではgithub_sshというファイル名で書いてあります<br>
+-- env.json<br>
+-- Github-Clone-Push-file.ipynb(このファイル)
</p>
<p><font size="3">となっています</p>

<p><font size="3">JSONファイルの想定フォーマットは一番下にサンプルを記載してあります </p>

<p><font size="5" color="orange">1. 環境変数の設定</p>


In [1]:
%env REPO_NAME= ventilator-pressure-prediction
# <=====　ここの値を現在のリポジトリ名(=kaggleコンペ名)に変更！！！

import os
import json

REPO_NAME = os.environ['REPO_NAME'] #%cdコマンド内では $ で環境変数ではなく、Pythonで定義された変数にアクセスするため代入しておく
!echo $REPO_NAME


%cd /content/drive/MyDrive/kaggle/works

# 環境変数の設定
# Jupyterでは環境変数設定で　$ !export は使えないので注意
json_open = open('./env.json', 'r')
json_load = json.load(json_open)

os.environ['USERNAME'] = json_load['user']['username']
os.environ['USEREMAIL'] = json_load['user']['useremail']

os.environ['PASS'] = json_load['secrets']['pass']

env: REPO_NAME=ventilator-pressure-prediction
ventilator-pressure-prediction
/content/drive/MyDrive/kaggle/works



# <p><font size="5">2. Cloneリポジトリ<p>
<p><font size="5">3. Kaggleからデータセットのダウンロード</p>

<p><font size="3" color="red">初回のみ必要な操作です</p>

In [ ]:
# GitHubにSSH接続するために必要な作業作業
# ./.ssh/github_ssh を配置しておく配置しておく
# Github側に./.ssh/github_ssh.pub の中身を登録しておく
# KnownHostにgithub.comを登録する

%cd /content/drive/MyDrive/kaggle/works/

!cp -r ./.ssh /root/
!sudo chmod 600 /root/.ssh/github_ssh
!sudo chmod 600 /root/.ssh/github_ssh.pub

!ssh-keyscan -t rsa github.com >> /root/.ssh/known_hosts

# github.com:22 SSH-2.0-babeld-82e76f9b


In [ ]:
%cd /content/drive/MyDrive/kaggle/works

!git config --global user.name $USERNAME
!git config --global user.email $USEREMAIL

!git clone git@github.com:goya5858/$REPO_NAME

/content/drive/MyDrive/kaggle/works
Cloning into 'ventilator-pressure-prediction'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 18 (delta 2), reused 9 (delta 0), pack-reused 0
Receiving objects: 100% (18/18), 6.70 KiB | 685.00 KiB/s, done.
Resolving deltas: 100% (2/2), done.


<p><font size=4 color=orangered>ファイル数が多いとGoogle Driveへの反映に大分時差が出ることがあるので注意!!</p>

In [ ]:
# データのダウンロード

from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth


auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

filename = "/root/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Download 100%.


In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle
#↑これやらないとデータひとつづつをダウンロードする地獄を見るよ!!

!kaggle competitions download -c $REPO_NAME \
--path /content/drive/MyDrive/kaggle/works/$REPO_NAME/input/$REPO_NAME

%cd /content/drive/MyDrive/kaggle/works/$REPO_NAME/input/$REPO_NAME
# 解凍先のdirに移動する　unzipの -d オプションで解凍先のディレクトリを指定した場合、ファイル数が多すぎるとバグるので、先に移動しておく
!unzip -o "/content/drive/MyDrive/kaggle/works/$REPO_NAME/input/$REPO_NAME/$REPO_NAME.zip" 
%cd /content/drive/MyDrive/kaggle/works/
!rm -f "/content/drive/MyDrive/kaggle/works/$REPO_NAME/input/$REPO_NAME/$REPO_NAME.zip"

  Using cached kaggle-1.5.12-py3-none-any.whl
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12
 94% 209M/223M [00:01<00:00, 122MB/s]
100% 223M/223M [00:02<00:00, 112MB/s]
/content/drive/MyDrive/kaggle/works/ventilator-pressure-prediction/input/ventilator-pressure-prediction
Archive:  /content/drive/MyDrive/kaggle/works/ventilator-pressure-prediction/input/ventilator-pressure-prediction/ventilator-pressure-prediction.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               
/content/drive/MyDrive/kaggle/works


# <p><font size="5" color="orange">4. Pull</p>
<p><font color=orangered>現在のDrive内のリポジトリの状態を無視して強制Pullします</p>

In [18]:
# SSH接続のためのファイルをルートディレクトリにコピー
%cd /content/drive/MyDrive/kaggle/works/
!cp -r ./.ssh /root/

%cd /content/drive/MyDrive/kaggle/works/$REPO_NAME

!git config --global user.name $USERNAME
!git config --global user.email $USEREMAIL

# ローカルの内容に未保存の変更があったとしても、強制的にリモートに同期する
!git fetch origin
!git reset --hard origin/main

/content/drive/MyDrive/kaggle/works
/content/drive/MyDrive/kaggle/works/ventilator-pressure-prediction
HEAD is now at 04b6764 from mac-onpre


# <p><font size="5" color="orange">5. Commit & Push</p>

In [20]:
#　変更されたファイルをaddして、ローカルリポジトリにコミット
%cd /content/drive/MyDrive/kaggle/works/$REPO_NAME

!git config --global user.name $USERNAME
!git config --global user.email $USEREMAIL

!git add .
!git commit -m 'from colab' 

/content/drive/MyDrive/kaggle/works/ventilator-pressure-prediction
[main 9e47ebe] from colab
 2 files changed, 1 insertion(+), 2 deletions(-)
 rewrite working/EDA2.ipynb (88%)
 delete mode 160000 working/rapidsai-csp-utils


In [21]:
# リモートリポジトリにプッシュ
# remote set-urlで、Githubのアカウントとパスワードを設定する必要がある
# ここのPASSWORDどうにかしたいなぁ
%cd /content/drive/MyDrive/kaggle/works/$REPO_NAME

!git remote set-url origin git@github.com:goya5858/$REPO_NAME.git
!git push origin main #ブランチ名は適宜変更

/content/drive/MyDrive/kaggle/works/ventilator-pressure-prediction
Counting objects: 8, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (8/8), done.
Writing objects: 100% (8/8), 6.11 KiB | 271.00 KiB/s, done.
Total 8 (delta 6), reused 0 (delta 0)
remote: Resolving deltas: 100% (6/6), completed with 3 local objects.
To github.com:goya5858/ventilator-pressure-prediction.git
   04b6764..9e47ebe  main -> main


# JSONファイル記載例
コピペをして該当部分を書き換えてください
ファイル名は　env.json　を想定しています

In [ ]:
{
    "user":{
        "username":"USERNAME for Github",
        "useremail": "EMAIL for Github"
    },

    "secrets":{
        "pass": "PASSWORD for Github"
    }
}